List Classes
====================
A julia version of `list_classes.mm`

In [1]:
using ObjectiveC


In [ ]:
run(`git show --oneline -s`)
nothing

0ae5145 use notebooks, begin look at structure of frameworks


In [2]:
ObjectiveC.load_framework("Foundation")


true

In [3]:
num_classes = ccall(:objc_getClassList, Cint, (Ptr{Cvoid}, Cint), C_NULL, 0)


2131

In [4]:
classes = Array{Ptr{Cvoid}}(undef, num_classes)
num_classes = ccall(:objc_getClassList, Cint, (Ptr{Cvoid}, Cint), classes, num_classes)


2131

In [5]:
names = [ccall(:class_getName, Ptr{Cchar}, (Ptr{Cvoid},),c) |> unsafe_string for c in classes] 


2131-element Vector{String}:
 "__NSGenericDeallocHandler"
 "__NSAtom"
 "_NSZombie_"
 "__NSMessageBuilder"
 "NSProxy"
 "_LSCoreTypesRecordProxy"
 "NSUndoManagerProxy"
 "NSProtocolChecker"
 "NSConcreteProtocolChecker"
 "NSAutoContentAccessingProxy"
 ⋮
 "OS_xpc_double"
 "OS_xpc_bool"
 "OS_xpc_null"
 "OS_xpc_session"
 "OS_xpc_service"
 "OS_xpc_connection"
 "__NSUnrecognizedTaggedPointer"
 "Protocol"
 "__IncompleteProtocol"

In [6]:
images = [ccall(:class_getImageName, Ptr{Cchar}, (Ptr{Cvoid},),c) |> unsafe_string for c in classes] 


2131-element Vector{String}:
 "/System/Library/Frameworks/CoreFoundation.framework/Versions/A/CoreFoundation"
 "/System/Library/Frameworks/CoreFoundation.framework/Versions/A/CoreFoundation"
 "/System/Library/Frameworks/CoreFoundation.framework/Versions/A/CoreFoundation"
 "/System/Library/Frameworks/CoreFoundation.framework/Versions/A/CoreFoundation"
 "/System/Library/Frameworks/Foundation.framework/Versions/C/Foundation"
 "/System/Library/Frameworks/Core" ⋯ 60 bytes ⋯ "ework/Versions/A/LaunchServices"
 "/System/Library/Frameworks/Foundation.framework/Versions/C/Foundation"
 "/System/Library/Frameworks/Foundation.framework/Versions/C/Foundation"
 "/System/Library/Frameworks/Foundation.framework/Versions/C/Foundation"
 "/System/Library/Frameworks/Foundation.framework/Versions/C/Foundation"
 ⋮
 "/usr/lib/system/libxpc.dylib"
 "/usr/lib/system/libxpc.dylib"
 "/usr/lib/system/libxpc.dylib"
 "/usr/lib/system/libxpc.dylib"
 "/usr/lib/system/libxpc.dylib"
 "/usr/lib/system/libxpc.dylib"
 "/us

In [7]:
class = C_NULL
start = time_ns()
for n in names
    class = ccall(:objc_getClass, Ptr{Cvoid}, (Ptr{Cchar},),n)
end
fin = time_ns()

print("Lookup by name: $((fin-start)/1000)us Total, $((fin-start)/num_classes)ns per Class")


Lookup by name: 10000.833us Total, 4693.023463162835ns per Class

In [8]:
using BenchmarkTools


In [9]:
@benchmark ccall(:objc_getClass, Ptr{Cvoid}, (Ptr{Cchar},),$(names[10]))

BenchmarkTools.Trial: 10000 samples with 985 evaluations.
 Range (min … max):  52.834 ns … 169.289 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     54.611 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   55.192 ns ±   2.418 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▄▂      ▆█▄▁   ▁ ▂▅▄▂▁  ▂                                    ▁
  ██▆▄▅▅▅▅████▇█████████████▆▅▆▄▆▅▆▆▄▅▄▄▄▅▄▄▃▃▃▃▃▃▅▄▃▅▃▃▃▄▃▃▃▃ █
  52.8 ns       Histogram: log(frequency) by time      64.1 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [10]:
## wrap lookup in function
function lookup(names::Vector{String})
    class = C_NULL
    start = time_ns()
    for n in names
        class = ccall(:objc_getClass, Ptr{Cvoid}, (Ptr{Cchar},),n)
    end
    fin = time_ns()
    return start, fin
end
start, fin = lookup(names)

print("Lookup by name: $((fin-start)/1000)us Total, $((fin-start)/num_classes)ns per Class")

Lookup by name: 387.0us Total, 181.60488033786953ns per Class

In [12]:
using DataFrames

In [13]:
df = DataFrame(;name=names, image=images)

Row,name,image
,String,String
1,__NSGenericDeallocHandler,/System/Library/Frameworks/CoreFoundation.framework/Versions/A/CoreFoundation
2,__NSAtom,/System/Library/Frameworks/CoreFoundation.framework/Versions/A/CoreFoundation
3,_NSZombie_,/System/Library/Frameworks/CoreFoundation.framework/Versions/A/CoreFoundation
4,__NSMessageBuilder,/System/Library/Frameworks/CoreFoundation.framework/Versions/A/CoreFoundation
5,NSProxy,/System/Library/Frameworks/Foundation.framework/Versions/C/Foundation
6,_LSCoreTypesRecordProxy,/System/Library/Frameworks/CoreServices.framework/Versions/A/Frameworks/LaunchServices.framework/Versions/A/LaunchServices
7,NSUndoManagerProxy,/System/Library/Frameworks/Foundation.framework/Versions/C/Foundation
8,NSProtocolChecker,/System/Library/Frameworks/Foundation.framework/Versions/C/Foundation
9,NSConcreteProtocolChecker,/System/Library/Frameworks/Foundation.framework/Versions/C/Foundation


In [24]:
f(fu) = (args...; kw...) -> x -> fu(x, args...; kw...
)

f (generic function with 1 method)

In [28]:
image = df |> f(groupby)(:image) |> f(combine)(nrow) |> f(sort)(order(:nrow, rev=true))

Row,image,nrow
,String,Int64
1,/System/Library/Frameworks/Foundation.framework/Versions/C/Foundation,796
2,/System/Library/Frameworks/Network.framework/Versions/A/Network,241
3,/System/Library/Frameworks/CoreServices.framework/Versions/A/Frameworks/LaunchServices.framework/Versions/A/LaunchServices,176
4,/System/Library/Frameworks/SecurityFoundation.framework/Versions/A/SecurityFoundation,161
5,/System/Library/Frameworks/CFNetwork.framework/Versions/A/CFNetwork,159
6,/System/Library/Frameworks/CoreFoundation.framework/Versions/A/CoreFoundation,157
7,/System/Library/Frameworks/Security.framework/Versions/A/Security,66
8,/System/Library/PrivateFrameworks/CoreAutoLayout.framework/Versions/A/CoreAutoLayout,49
9,/usr/lib/swift/libswiftCore.dylib,41


In [30]:
"/System/Library/Frameworks/Foundation.framework/Versions/C/Foundation" ∈ image[:, :image]

true

In [31]:
"/usr/lib/libSystem.B.dylib" ∈ image[:,:image]

false